In [11]:
!pip install git+https://github.com/google/BIG-bench.git

  Cloning https://github.com/google/BIG-bench.git to /tmp/pip-req-build-yl2k6qkw
  Running command git clone --filter=blob:none --quiet https://github.com/google/BIG-bench.git /tmp/pip-req-build-yl2k6qkw
  Resolved https://github.com/google/BIG-bench.git to commit 092b196c1f8f14a54bbc62f24759d43bde46dd3b
  Preparing metadata (setup.py) ... done
Processing //tmp/pip-req-build-yl2k6qkw/bleurt/bleurt-b610120347ef22b494b6d69b4316e303f5932516.zip (from bigbench==0.0.1)
  Preparing metadata (setup.py) ... done
  Created wheel for bleurt: filename=BLEURT-0.0.2-py3-none-any.whl size=16454005 sha256=0e7f3eda63d24c2c81feac76b7b9d90a214fd865c2919f98b92237ecebb98b65
  Stored in directory: /root/.cache/pip/wheels/fd/b1/86/350a38c08511c851a30521471649ed8f3fb9eeec42b8b8a2ab
Successfully built bleurt
  Attempting uninstall: bleurt
    Found existing installation: BLEURT 0.0.2
    Uninstalling BLEURT-0.0.2:
      Successfully uninstalled BLEURT-0.0.2


In [12]:
!pip install 'datasets<3.0.0' 'httpx==0.27.2'

In [13]:
!pip install cohere

In [14]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import os
import seqio
import tensorflow_datasets as tfds
from bigbench.bbseqio import task_api
from bigbench.bbseqio import tasks
import re
import string
import cohere
import openai
from google.colab import userdata
import json
import time

In [15]:
bb_mix = seqio.get_mixture_or_task("bigbench:all_json.mix.t5_default_vocab.0_shot.all_examples")

print("SeqIO Mixture:", bb_mix.name)

print("\nNumber of SeqIO Tasks in this Mixture:", len(bb_mix.tasks))
print("\nExample Subtasks:")
all_subtasks = sorted([t.name for t in bb_mix.tasks])
# print("\n".join(all_subtasks[:7]) + "\n...")
print("\n".join([subtask for subtask in all_subtasks if "causal" in subtask]))

SeqIO Mixture: bigbench:all_json.mix.t5_default_vocab.0_shot.all_examples

Number of SeqIO Tasks in this Mixture: 1762

Example Subtasks:
bigbench:causal_judgment.mul.t5_default_vocab.0_shot.all_examples


In [1]:
print("Q: When I'm full, I'll be able to point the way. Nothing moves me when I'm empty. I have two skins: one that is outside and one that is within. What exactly am I?\n  choice: covering\n  choice: triangle\n  choice: glove\n  choice: band aid\n  choice: index finger\nA:")

Q: When I'm full, I'll be able to point the way. Nothing moves me when I'm empty. I have two skins: one that is outside and one that is within. What exactly am I?
  choice: covering
  choice: triangle
  choice: glove
  choice: band aid
  choice: index finger
A:


In [16]:
task = seqio.get_mixture_or_task("bigbench:riddle_sense.mul.t5_default_vocab.0_shot.all_examples")

sequence_length = {"inputs": 32, "targets": 32}
ds = task.get_dataset(
    sequence_length=sequence_length,  #  Trim long token sequences.
    split="all",  # Available splits = "all", "train", "validation".
)
for d in tfds.as_numpy(ds.take(5)):
  print(d)

{'inputs_pretokenized': b"Q: You may be in a room full of flowers in one case, but you'll never step into it in another. What makes you say that?\n  choice: funeral\n  choice: windowsill\n  choice: leg\n  choice: disparagement\n  choice: vase\nA:", 'inputs': array([1593,   10,  148,  164,   36,   16,    3,    9,  562,  423,   13,
       3652,   16,   80,  495,    6,   68,   25,   31,  195,  470, 1147,
        139,   34,   16,  430,    5,  363,  656,   25,  497,   24],
      dtype=int32), 'idx': array([8, 1], dtype=int32), 'targets_pretokenized': b'leg', 'targets': array([4553], dtype=int32), 'scores': 0, 'answers': array([b'funeral'], dtype=object)}
{'inputs_pretokenized': b"Q: When I'm full, I'll be able to point the way. Nothing moves me when I'm empty. I have two skins: one that is outside and one that is within. What exactly am I?\n  choice: covering\n  choice: triangle\n  choice: glove\n  choice: band aid\n  choice: index finger\nA:", 'inputs': array([1593,   10,  366,   27,   31,

In [17]:
task = seqio.get_mixture_or_task("bigbench:causal_judgment.mul.t5_default_vocab.0_shot.all_examples")

sequence_length = {"inputs": 32, "targets": 32}
ds_cj = task.get_dataset(
    sequence_length=sequence_length,  #  Trim long token sequences.
    split="all",  # Available splits = "all", "train", "validation".
)
for d in tfds.as_numpy(ds_cj.take(5)):
  print(d)

{'inputs_pretokenized': b"How would a typical person answer each of the following questions about causation?\n\n\nQ: Billy and Suzy inherited an unusual type of hybrid car that has two special car batteries called Bartlett batteries. The car won't start unless it has at least one Bartlett battery. Having a second Bartlett battery isn't necessary to start the car. When they got the car, both Bartlett batteries were missing. One day, Billy and Suzy are both out of the house. Billy is visiting his friend's house, and notices that his friend has a Bartlett battery. Billy asks his friend to sell the battery to him, and his friend says that he's willing to sell it for a fair price, so Billy buys the Bartlett battery from his friend. Meanwhile, on the other side of town, Suzy walks into an automotive parts shop and happens to notice that they have a single Bartlett battery in stock. Suzy decides to buy the Bartlett battery from the shop. When Billy and Suzy get home, they installed the two Ba

In [18]:
task = seqio.get_mixture_or_task("bigbench:odd_one_out.mul.t5_default_vocab.0_shot.all_examples")

sequence_length = {"inputs": 32, "targets": 32}
ds_ooo = task.get_dataset(
    sequence_length=sequence_length,  #  Trim long token sequences.
    split="all",  # Available splits = "all", "train", "validation".
)
for d in tfds.as_numpy(ds_ooo.take(5)):
  print(d)

{'inputs_pretokenized': b'Q: Pick the odd word out: jump, clap, pencil, read, run\nA:', 'inputs': array([ 1593,    10,  8356,     8,  8617,  1448,    91,    10,  4418,
           6,     3,  4651,   102,     6, 13966,     6,   608,     6,
         661,    71,    10], dtype=int32), 'idx': array([7, 2], dtype=int32), 'targets_pretokenized': b'pencil', 'targets': array([13966], dtype=int32), 'scores': 1, 'answers': array([b'pencil'], dtype=object)}
{'inputs_pretokenized': b'Q: Pick the odd word out: bedroom, kitchen, carpet, dining room, bathroom\nA:', 'inputs': array([1593,   10, 8356,    8, 8617, 1448,   91,   10, 2923,    6, 1228,
          6, 4898,    6, 2851,  562,    6, 2582,   71,   10], dtype=int32), 'idx': array([40,  3], dtype=int32), 'targets_pretokenized': b'dining room', 'targets': array([2851,  562], dtype=int32), 'scores': 0, 'answers': array([b'carpet'], dtype=object)}
{'inputs_pretokenized': b'Q: Pick the odd word out: apple, pear, strawberry, phone, banana\nA:', 'inputs':

In [26]:
def transform_question_string(question, choices):
    choices = [string.ascii_uppercase[i] + ": " + choice for i,choice in enumerate(choices)]
    inputs = question + "\n" + "\n".join(choices) + " \n" + "Answer: "

    return inputs

In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load model and tokenizer
deberta_model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base")
deberta_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def get_deberta_response(question, choices):

    inputs = transform_question_string(question, choices)
    encodings = deberta_tokenizer(inputs, padding=True, return_tensors="pt")
    outputs = deberta_model(**encodings)
    scores = outputs.logits.softmax(dim=-1).detach().numpy()
    best_choice_idx = scores.argmax(axis=0)[1]
    return choices[best_choice_idx]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [23]:
co = cohere.ClientV2(api_key=userdata.get('COHERE_API_KEY'))

def get_cohere_response(question, choices, **kwargs):

    prompt = transform_question_string(question, choices)
    system_prompt = """
                    You are a helpful assistant. You have to solve the given common sense questions in the prompt.
                    After the questions there will be choices/options that you have to choose from.
                    Think logically before answering each question.
                    Output your response in this template
                    choice_letter_of_correct_answer: answer_corresponding_to_the_letter
                    For example:
                        C: land

                    Follow the template of response strictly. Choose your answer from the list choices given after the question.
                    Each choice will be preceeded by an uppercase letter
                    """
    model="command-r-08-2024"

    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": prompt}]


    kwargs['model'] = model
    kwargs['messages'] = messages


    response = co.chat(**kwargs)

    return response.message.content[0].text


In [27]:
q = "Before you can have me, I need to be taken. What am I?"
c = ['beg', 'picture', 'move', 'become adult', 'paper']

transform_question_string(q,c)

'Before you can have me, I need to be taken. What am I?\nA: beg\nB: picture\nC: move\nD: become adult\nE: paper \nAnswer: '

In [25]:
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def get_gpt_response(question, choices, model_num='4-o-mini', **kwargs):

    prompt = transform_question_string(question, choices)
    system_prompt = """
                    You are a helpful assistant. You have to solve the given common sense questions in the prompt.
                    After the questions there will be choices/options that you have to choose from.
                    Think logically before answering each question.
                    Output your response in this template
                    choice_letter_of_correct_answer: answer_corresponding_to_the_letter
                    For example:
                        C: land

                    Follow the template of response strictly.
                    """

    if model_num == "3.5":
        model="gpt-3.5-turbo"
    elif model_num == "4-o-mini":
        model="gpt-4o-mini"
    messages=[{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}]

    kwargs["temperature"] = 0.0
    kwargs["top_p"] = 1
    kwargs["frequency_penalty"] = 0.0
    kwargs["presence_penalty"] = 0.0
    kwargs["timeout"] = 4*10*60  # 40 minutes
    kwargs["model"] = model
    kwargs["messages"] = messages


    response = client.chat.completions.create(**kwargs)

    return response.choices[0].message.content.strip()


In [22]:
for example in tfds.as_numpy(ds_cj.take(5)):
    full_question = example['inputs_pretokenized'].decode("utf-8")
    # question = re.findall("Q:\s(.*)\s+choice:(\s.*){3,}", full_question)[0][0]
    # question = "Pick the odd word out:"
    question = re.findall("Q:\s(.*)\s.*", full_question)[0]
    # choices = re.findall("choice:\s(.*)", full_question)
    # choices = re.findall(f"{question}\s(.*)", full_question)[0].split(',')
    choices = ["Yes", "No"]
    correct_answer = example['answers'][0].decode()
    # print(question)
    # print(full_question)

    # deberta_response = get_deberta_response(question, choices)
    # gpt3_5_response = get_gpt_response(question, choices, model_num='3.5')
    # gpt4_o_response = get_gpt_response(question, choices, model_num='4-o-mini')
    # command_r_response = get_cohere_response(question, choices)

    print(f"Question: {question}")
    print(f"Choices: {choices}")
    print(f"Correct Answer: {correct_answer}")
    # print(f"DeBERTa Response: {deberta_response}")
    # print(f"GPT-3.5 Response: {gpt3_5_response}")
    # print(f"GPT-4o Response: {gpt4_o_response}")
    # print(f"Command-R Response: {command_r_response}")
    print("-" * 50)

Question: John is standing near the tracks of a trolley. John notices that the brakes of the trolley have failed. Five workmen are working on the tracks with their backs turned. John sees that the runaway trolley is headed for the five workmen who will be killed if it proceeds on its present course. The only way to save these five workmen is to hit a switch that will turn the trolley onto the side tracks. Moreover, there is a dog on the tracks with its back turned. John knows that the five workmen and the dog will be saved if he hits the switch. John thinks 'I don't care at all about saving the dog. I just want to save the five workmen.' John decides to hit the switch. Sure enough, the trolley turns on the side tracks, the five workmen and the dog on the main tracks are saved. Did John intentionally save the dog?
Choices: ['Yes', 'No']
Correct Answer: No
--------------------------------------------------
Question: Prof. Smith works at a large university. At this university, in order to

In [30]:
def zero_shot_responses(ds, savefile, save=False):

    responses = []
    start = time.time()
    minute = time.time()

    for i, example in enumerate(tfds.as_numpy(ds)):
        full_question = example['inputs_pretokenized'].decode("utf-8")
        # question = re.findall("Q:\s(.*)\s+choice:(\s.*){3,}", full_question)[0][0]
        # question = "Pick the odd word out:"
        question = re.findall("Q:\s(.*)\s.*", full_question)[0]
        # choices = re.findall("choice:\s(.*)", full_question)
        # choices = re.findall(f"{question}\s(.*)", full_question)[0].split(',')
        choices = ["Yes", "No"]
        correct_answer = example['answers'][0].decode('utf-8')

        deberta_response = get_deberta_response(question, choices)
        gpt3_5_response = get_gpt_response(question, choices, model_num='3.5')
        gpt4_o_response = get_gpt_response(question, choices, model_num='4-o-mini')

        try:
            command_r_response = get_cohere_response(question, choices)
        except:
            print("Too many responses in a minute")
            time.sleep(60)
            try:
                command_r_response = get_cohere_response(question, choices)
            except:
                print(f"skipped index {i}")

        end = time.time()
        time_elapsed = end-start

        response = {
            "question": question,
            "choices": choices,
            "answer": correct_answer,
            "deberta": deberta_response,
            "gpt3.5": gpt3_5_response,
            "gpt4o": gpt4_o_response,
            "commandr": command_r_response
        }

        responses.append(response)


        if i%20 == 0:
            print(f"Generated [{i}/{len(ds)}] responses  |  Time elapses: {time_elapsed:.2f}")

        if save:
            with open(savefile, "a", encoding='utf-8') as f:
                f.write(json.dumps(response) + '\n')

    print(f"Responses saved to {savefile}")


    return responses

responses = zero_shot_responses(ds_cj, "zero_shot_responses_causal_judgement.jsonl", save=True)

Generated [0/380] responses  |  Time elapses: 4.07
Generated [20/380] responses  |  Time elapses: 56.33
Generated [40/380] responses  |  Time elapses: 103.21
Generated [60/380] responses  |  Time elapses: 154.24
Generated [80/380] responses  |  Time elapses: 198.32
Generated [100/380] responses  |  Time elapses: 245.08
Generated [120/380] responses  |  Time elapses: 286.73
Generated [140/380] responses  |  Time elapses: 339.73
Generated [160/380] responses  |  Time elapses: 393.34
Generated [180/380] responses  |  Time elapses: 436.86
Generated [200/380] responses  |  Time elapses: 486.53
Generated [220/380] responses  |  Time elapses: 529.74
Generated [240/380] responses  |  Time elapses: 572.21
Generated [260/380] responses  |  Time elapses: 615.86
Generated [280/380] responses  |  Time elapses: 661.17
Generated [300/380] responses  |  Time elapses: 706.78
Generated [320/380] responses  |  Time elapses: 747.03
Generated [340/380] responses  |  Time elapses: 798.92
Generated [360/380]

### Test on finetuned models

In [38]:
import torch
from transformers import AutoTokenizer, AutoModelForMultipleChoice


In [39]:
tokenizer = AutoTokenizer.from_pretrained("./deberta_finetuned/")
model = AutoModelForMultipleChoice.from_pretrained("./deberta_finetuned/")

In [41]:
def predict_answer(model, tokenizer, question, choices):
    # Prepare inputs similar to how they were prepared during training
    # Each choice forms a separate input sequence
    inputs = [f"Question: {question} Choice: {choice}" for choice in choices]

    # Tokenize all choice sequences
    encoding = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

    # The model expects inputs in shape: (batch_size, num_choices, seq_length)
    # Here we have one "batch" with multiple choices, so we add a batch dimension
    for key in encoding:
        encoding[key] = encoding[key].unsqueeze(0)  # Now shape: (1, num_choices, seq_length)

    # Move model and inputs to appropriate device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    encoding = {k: v.to(device) for k, v in encoding.items()}

    # Predict without gradient updates
    with torch.no_grad():
        outputs = model(**encoding)

    # The model outputs logits of shape (batch_size=1, num_choices)
    # Choose the answer with the highest logit score
    predicted_label = outputs.logits.argmax(dim=1).item()
    predicted_answer = choices[predicted_label]

    return predicted_answer



In [59]:
import pandas as pd

responses = pd.read_json('zero_shot_responses_oddoneout.jsonl', lines=True)

responses.head()

,question,choices,answer,deberta,gpt3.5,gpt4o,commandr
0,Pick the odd word out:,"[square, circle, triangle, trousers, recta...",trousers,square,D: trousers,D: trousers,D: trousers
1,Pick the odd word out:,"[Spain, France, German, England, Singapore]",German,Spain,E: Singapore,C: German,E: Singapore
2,Pick the odd word out:,"[dentist, doctor, postman, hospital, cashier]",hospital,dentist,A: dentist,C: postman,C: postman
3,Pick the odd word out:,"[milk, pink, coffee, tea, beer]",pink,milk,A: milk,B: pink,D: tea
4,Pick the odd word out:,"[Albania, Asia, Africa, America]",Albania,Albania,B: Asia,A: Albania,B: Asia


In [54]:
s = responses.sample(1).iloc[0]

In [58]:
s

,164
question,Pick the odd word out:
choices,"[break, play, do, write]"
answer,do
deberta,break
gpt3.5,A: break
gpt4o,A: break
commandr,D: write


In [57]:
q = "Pick the odd word out:"
c = ['break', ' play', ' do', ' write']
predict_answer(model, tokenizer, q, c)

' do'